In [ ]:
# !pip install torch torchvision timm transformers opencv-python
# !apt-get install ffmpeg -y

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from PIL import Image
from transformers import BertTokenizer
import os

In [3]:
import cv2
import os

def map_frames_to_text(video_path, annotations):
    # 비디오 파일 열기
    video_capture = cv2.VideoCapture(video_path)
    
    # FPS와 총 프레임 수 확인
    fps = video_capture.get(cv2.CAP_PROP_FPS)
    num_frames = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
    
    print(f"FPS: {fps}, Total Frames: {num_frames}")
    
    # 각 프레임을 시간에 맞춰 텍스트와 매핑
    frame_text_mapping = {}
    for frame in range(num_frames):
        time_sec = frame / fps  # 프레임을 시간(초)으로 변환
        for (start, end, text) in annotations:
            if start <= time_sec < end:
                frame_text_mapping[frame] = text
                break
        else:
            frame_text_mapping[frame] = ""  # 해당하는 텍스트가 없을 경우 빈 텍스트
            
    # 비디오 캡처 객체 해제
    video_capture.release()
    
    return frame_text_mapping

In [ ]:
frame_text_mapping = map_frames_to_text(video_path, annotations)

for frame, text in frame_text_mapping.items():
    print(f"Frame {frame}: {text}")

In [ ]:
# 2. 데이터셋 정의
class LipReadingDataset(Dataset):
    def __init__(self, video_paths, annotations, 
                 tokenizer, transform, sequence_length=5):
        self.video_paths = video_paths
        self.annotations = annotations
        self.tokenizer = tokenizer
        self.transform = transform
        self.sequence_length = sequence_length  # 묶을 프레임 수
        
        # 프레임과 텍스트 매핑 생성
        self.frame_text_mapping = []
        for video_annotation in annotations:
            num_frames = len(video_paths[0])  # 각 비디오의 프레임 수
            mapping = map_frames_to_text(video_paths[0], video_annotation)
            self.frame_text_mapping.append(mapping)

    def __len__(self):
        return sum([len(v) for v in self.video_paths])

    def __getitem__(self, idx):
        # 비디오 인덱스 및 프레임 인덱스 추출
        video_idx, frame_idx = self.get_video_frame_index(idx)
        frames = []
        texts = []
        
        # 남은 프레임이 sequence_length보다 적다면 가능한 만큼만 가져오기
        remaining_frames = len(self.video_paths[video_idx]) - frame_idx
        effective_sequence_length = min(self.sequence_length, remaining_frames)

        # 프레임 처리
        for i in range(effective_sequence_length):
            frame_path = self.video_paths[video_idx][frame_idx + i]
            frame = preprocess_video_frame(frame_path, self.transform)
            frames.append(frame)

            # 해당 프레임에 대응하는 텍스트
            text = self.frame_text_mapping[video_idx][frame_idx + i]
            texts.append(text)
        
        # 부족한 프레임 패딩 (여기서는 0으로 패딩 처리)
        for _ in range(self.sequence_length - effective_sequence_length):
            frames.append(torch.zeros_like(frames[0]))  # 프레임 크기에 맞춰 0으로 패딩
            texts.append("")  # 빈 텍스트 패딩

        # 여러 프레임의 텍스트를 연결하거나, 적절한 방식으로 병합
        combined_text = " ".join(texts)
        tokens = tokenize_text(combined_text, self.tokenizer)

        return {
            'video': torch.stack(frames),  # (sequence_length, C, H, W)
            'input_ids': tokens['input_ids'].squeeze(0),  # (max_length,)
            'attention_mask': tokens['attention_mask'].squeeze(0)
        }

    def get_video_frame_index(self, idx):
        """비디오 인덱스와 프레임 인덱스를 계산"""
        cum_frames = 0
        for i, video in enumerate(self.video_paths):
            if idx < cum_frames + len(video):
                return i, idx - cum_frames
            cum_frames += len(video)
        raise IndexError("Index out of range")

In [ ]:
# 3. 전처리 함수 정의
def preprocess_video_frame(frame_path, transform):
    img = Image.open(frame_path).convert('RGB')
    img_tensor = transform(img)
    return img_tensor

def tokenize_text(text, tokenizer):
    tokens = tokenizer(text, return_tensors='pt', padding='max_length', truncation=True, max_length=50)
    return tokens

In [ ]:
# 4. 모델 정의
import timm

class Frontend(nn.Module):
    def __init__(self, model_type="convnext"):
        super(Frontend, self).__init__()
        if model_type == "convnext":
            self.model = timm.create_model('convnext_base', pretrained=True)
        self.model.reset_classifier(0)

    def forward(self, x):
        return self.model(x)

class Backbone(nn.Module):
    def __init__(self, model_type="swin"):
        super(Backbone, self).__init__()
        if model_type == "swin":
            self.model = timm.create_model('swin_base_patch4_window7_224', pretrained=True)
        self.model.reset_classifier(0)

    def forward(self, x):
        return self.model(x)

class LipReadingModel(nn.Module):
    def __init__(self, frontend_type="convnext", backend_type="swin"):
        super(LipReadingModel, self).__init__()
        self.frontend = Frontend(model_type=frontend_type)
        self.backend = Backbone(model_type=backend_type)
        self.fc = nn.Linear(1024, 512)
        
    def forward(self, x):
        x = self.frontend(x)
        x = self.backend(x)
        x = self.fc(x)
        return x

In [ ]:
# 5. 학습 코드
def train_model(model, dataloader, criterion, optimizer, device, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for batch in dataloader:
            videos = batch['video'].to(device)
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            outputs = model(videos)
            labels = torch.randint(0, 1000, (outputs.size(0),)).to(device)  # 임의 레이블 (추후 수정 필요)
            
            loss = criterion(outputs, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        avg_loss = running_loss / len(dataloader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

In [ ]:
# # 6. 데이터 경로 및 실제 사용 예시
# video_paths = [
#     [f'./videos/video1/frame_{i}.png' for i in range(180)],  # 첫 번째 비디오의 프레임 경로들
#     [f'./videos/video2/frame_{i}.png' for i in range(200)],  # 두 번째 비디오의 프레임 경로들
# ]

# annotations = [
#     [(0, 2, "Hello"), (2, 4, "How are you?"), (4, 6, "I am fine")],  # 첫 번째 비디오 텍스트 주석
#     [(0, 1, "Good morning"), (1, 3, "Have a nice day")],  # 두 번째 비디오 텍스트 주석
# ]

# transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# ])

# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# dataset = LipReadingDataset(video_paths, annotations, fps=30, tokenizer=tokenizer, transform=transform)

#TODO: 데이터셋 로컬 저장된 파일 가져오기
dataloader = DataLoader(dataset, batch_size=8, shuffle=True, num_workers=4)

model = LipReadingModel(frontend_type="convnext", backend_type="swin")
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# 학습 시작
train_model(model, dataloader, criterion, optimizer, device, num_epochs=10)

In [ ]:
# 7. 추론
def infer(model, video_frames, tokenizer, transform, device='cpu'):
    model.eval()  # 모델을 평가 모드로 설정 (학습하지 않음)
    
    # 프레임을 전처리
    processed_frames = [preprocess_video_frame(frame, transform) for frame in video_frames]
    video_tensor = torch.stack(processed_frames).to(device)  # 배치로 묶음

    with torch.no_grad():  # 추론 시에는 그래디언트 계산을 하지 않음
        outputs = model(video_tensor)
        # 예를 들어, 가장 높은 확률을 가진 클래스를 예측
        predicted_labels = torch.argmax(outputs, dim=1)
    
    return predicted_labels.cpu().numpy()

# 테스트용 추론 함수 예시
def load_test_video_frames(video_dir):
    # 예: 비디오 디렉토리에서 프레임 이미지 파일 로드
    return [f'{video_dir}/frame_{i}.png' for i in range(30)]

if __name__ == "__main__":
    # 학습된 모델을 로드한 후 추론 수행
    test_video_dir = './videos/video1'  # 예시로 추론할 비디오 디렉토리
    video_frames = load_test_video_frames(test_video_dir)  # 테스트 비디오의 프레임 경로

    # 추론 수행
    predictions = infer(model, video_frames, tokenizer, transform, device)
    print("Predictions:", predictions)
